In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.usb_camera import USBCamera
import traitlets
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import imutils
import time
import subprocess

In [ ]:
subprocess.call("sh bashWheel.sh 0", shell=True)
subprocess.call("sh bashTurn.sh 500", shell=True)

controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

In [ ]:
dead_zone_limit = 0.1
password = 'jetson'

leftRightVal = 0.0

def upDown(change):
    val = change['new']
    if val < -dead_zone_limit: # Up
#         subprocess.call("echo {} | sudo -S sh bashWheel.sh 1000".format(password), shell=True)
        subprocess.call("echo {} | sudo -S sh newForward.sh".format(password), shell=True)
    elif val > dead_zone_limit: # Down
#         print("Down")
#         subprocess.call("echo {} | sudo -S sh bashWheelBack.sh 1000".format(password), shell=True)
        subprocess.call("echo {} | sudo -S sh newReverse.sh".format(password), shell=True)
    elif round(val) == 0.0:
        subprocess.call("echo {} | sudo -S sh bashWheelBack.sh 0".format(password), shell=True)
#     print(val)
    
def leftRight(change):
    # 700 / 500 / 300
    val = change['new']
    adjust_val = (((val + 1) * 400) / 2) + 300
    if val > dead_zone_limit: # Right 350
        subprocess.call("echo {} | sudo -S sh bashTurn.sh {}".format(password, str(round(adjust_val))), shell=True)
    elif val < -dead_zone_limit: # Left 650
        subprocess.call("echo {} | sudo -S sh bashTurn.sh {}".format(password, str(round(adjust_val))), shell=True)
    elif round(val) == 0.0:
        subprocess.call("echo {} | sudo -S sh bashTurn.sh 500".format(password), shell=True)
    global leftRightVal
    leftRightVal = val
#     print(leftRightVal)
    
# Controlling throttle with C-Stick
controller.axes[5].observe(upDown, names='value')

# Controlling turns with Main Joystick
controller.axes[0].observe(leftRight, names='value')

In [ ]:
camera = USBCamera(width=360, height=360)
image = camera.read()
image_widget =  widgets.Image(format='jpeg')
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')
display(image_widget)

In [ ]:
global DATASET_DIR 
DATASET_DIR = 'Images'

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

counter = 20708
def save_snapshot(change):
    global counter
    if change['new']:
        image_path = os.path.join(DATASET_DIR, str(counter) + " "+ str(leftRightVal) + '.jpg')
        counter += 1
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[5].observe(save_snapshot, names='value')

In [ ]:
camera.stop()